In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/FaceNet/facenet_keras.h5' facenet_keras.h5
!cp '/content/drive/MyDrive/FaceNet/archive.zip' archive.zip
!cp -r '/content/drive/MyDrive/test' test
!cp -r '/content/drive/MyDrive/nonsense' nonsense

In [ ]:
!unzip archive.zip

In [ ]:
!sudo pip install imbalanced-learn

In [ ]:
from os import listdir
from os.path import isdir
from PIL import Image
from numpy import asarray

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.pipeline import Pipeline
from numpy import load, expand_dims

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# load pre-trained FaceNet model
from keras.models import load_model
facenet_model = load_model('facenet_keras.h5')
print(facenet_model.inputs)
print(facenet_model.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


In [ ]:
def image_preprocessing(filename):
  image = Image.open(filename)
  image = image.convert('RGB')  # input color of FaceNet
  image = image.resize((160,160)) # input dimension of FaceNet
  face_arr = asarray(image)
  return face_arr

In [ ]:
def load_marvel_face(subdir):
  face = list()
  for file in listdir(subdir):
    img_path = subdir + file
    face_arr = image_preprocessing(img_path)
    face.append(face_arr)
  return face

In [ ]:
def load_dataset(dir):
  print("Loading the dataset...")
  X, y = list(), list()
  for subdir in listdir(dir):
    path = dir + '/' + subdir + '/'
    if not isdir(path):
      print("Image path not found...")
      continue
    marvel_face = load_marvel_face(path)
    marvel_name = [subdir for _ in range(len(marvel_face))]
    print("Loaded {} images of {}".format(len(marvel_face), subdir))
    X.extend(marvel_face)
    y.extend(marvel_name)
  print("Successfully loaded the dataset...")
  return asarray(X), asarray(y)

In [ ]:
def face_embedding(model,face):
  face = face.astype('float32')
  mean, std = face.mean(), face.std()
  face = (face-mean)/std
  per = expand_dims(face, axis=0)
  yhat = model.predict(per)
  return yhat[0]

In [ ]:
def get_prediction(data):
  output = []
  for i in range(data.shape[0]):
    per = expand_dims(data[i], axis = 0)
    if max(my_model.predict_proba(per)[0])<0.8: # if class prob less than threshold of 0.4, then face is unknown
      output.append(5)
    else:
      pred = my_model.predict(per)
      output.append(pred[0])
  return output
def inverse_prediction(prediction):
  result = []
  for i in range(len(prediction)):
    if prediction[i] == 5:
      result.append("unknown")      
    else:
      hero = labeller.inverse_transform(prediction[i])
      result.append(hero)
  return result

In [ ]:
# Load data from dataset
avengers_face, avengers_name = load_dataset('cropped_images') 
print(avengers_face.shape, avengers_name.shape)

Loading the dataset...
Loaded 66 images of mark_ruffalo
Loaded 53 images of chris_hemsworth
Loaded 54 images of scarlett_johansson
Loaded 51 images of robert_downey_jr
Loaded 50 images of chris_evans
Successfully loaded the dataset...
(274, 160, 160, 3) (274,)


In [ ]:
# Obtain face embeddings using FaceNet Model on dataset
new_avengers = list()
for face in avengers_face:
  embed = face_embedding(facenet_model,face)
  new_avengers.append(embed)
asarray(new_avengers)

Cause: could not parse the source code of <function <lambda> at 0x7ff7e149e710>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7ff7e149e710>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7ff7e149e320>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7ff7e149e320>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7ff7e149e4d0>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could no

array([[ 0.2537611 , -1.0822445 ,  0.620365  , ...,  1.4066731 ,
        -0.13999994, -1.3223813 ],
       [ 0.55559886, -0.5058398 ,  0.6943772 , ...,  0.46842912,
        -0.4281253 , -0.77652323],
       [ 0.5699115 , -0.04720436,  0.7755244 , ...,  0.5880779 ,
        -0.03605543, -0.8967809 ],
       ...,
       [-0.5789609 ,  0.9173926 , -0.4548461 , ..., -0.89618623,
         2.5931993 , -0.07359681],
       [ 0.2548324 ,  0.8861177 , -0.5750268 , ..., -0.642449  ,
         2.1674335 , -0.3172723 ],
       [-0.04740684,  1.0700345 ,  0.8628357 , ..., -0.7687462 ,
         0.95541435,  1.2531831 ]], dtype=float32)

In [ ]:
# Transformers:
#Normalize the training data
normalize = Normalizer(norm='l2')
#Label encode the Target values, here, names of the marvel heroes
labeller = LabelEncoder()
new_avengers = normalize.transform(new_avengers)
avengers_name = labeller.fit_transform(avengers_name) 
oversample = SMOTE(sampling_strategy = 'not majority', random_state = 3)

In [ ]:
print(new_avengers.shape, len(avengers_name))
print(new_avengers[1])
print(avengers_name[:5])

(274, 128) 274
[ 0.05046561 -0.04594594  0.06307098 -0.07250116 -0.10203902  0.00067759
  0.16923039 -0.07576329  0.03182524  0.01535801 -0.0904481  -0.10881097
 -0.10275429 -0.15165363 -0.10564321  0.07248892 -0.20822802 -0.01403647
 -0.01755909 -0.15612786  0.12070401 -0.03375864 -0.03049926  0.06357411
  0.06833943 -0.01600829 -0.135281   -0.0754101  -0.00910086 -0.02287504
  0.01776128 -0.03185196  0.06187846  0.00613883  0.00791412 -0.13192701
 -0.02725204  0.00097572 -0.02665826  0.00114756  0.0679507   0.0758658
  0.17258589  0.08556296 -0.02836961  0.04105192  0.05169252  0.13170515
 -0.2038522  -0.10904858  0.03865273 -0.13552909 -0.07071769  0.01699245
  0.01476816 -0.01090955  0.00407652  0.08602078 -0.04200199  0.03531005
  0.07031708 -0.11831962 -0.11623927 -0.08359942 -0.0265741   0.04690154
 -0.06348915  0.07061914  0.02669872 -0.10229507  0.00354535 -0.01099752
 -0.08753023 -0.05835701 -0.1574461   0.03186925 -0.01211626  0.14925171
  0.08946069 -0.00471347  0.04834936 

In [ ]:
# Use SMOTE to oversample the imbalanced dataset
X, y = oversample.fit_resample(new_avengers, avengers_name)
print(X.shape, len(y))

(330, 128) 330


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [ ]:
# Divide dataset into train and test set in 80:20 ratio
train_face, test_face, train_name, test_name = train_test_split(X, y, test_size = 0.2, random_state = 3)
print(train_face.shape, train_name.shape)
print(test_face.shape, test_name.shape)

(264, 128) (264,)
(66, 128) (66,)


In [ ]:
# Create Model for face classification
my_model = SVC(kernel = 'linear', probability = True, verbose =  True)
my_model.fit(train_face, train_name)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
yhat_train = get_prediction(train_face)
yhat_test = get_prediction(test_face)

In [ ]:
print(yhat_train[:5])
print(train_name[:5])

[4, 1, 2, 2, 2]
[4 1 2 2 2]


In [ ]:
train_accuracy = accuracy_score(train_name, yhat_train)
test_accuracy = accuracy_score(test_name, yhat_test)
print("Training accuracy of the model is {0} %.".format(train_accuracy*100))
print("Testing accuracy of the model is {0} %.".format(test_accuracy*100))

Training accuracy of the model is 100.0 %.
Testing accuracy of the model is 100.0 %.


In [ ]:
test_img = image_preprocessing('nonsense/4.jpg')
test_face = face_embedding(facenet_model, test_img)
test = expand_dims(test_face, axis=0)
prob = my_model.predict_proba(test)
print(prob)
pred = get_prediction(expand_dims(test_face, axis=0))
print(pred)

[[7.99171243e-05 6.39560746e-07 1.38311488e-03 9.98528624e-01
  7.70417016e-06]]
[3]


In [ ]:
dir = 'test'
result = []
for file in listdir(dir):
  path = dir + '/' + file
  print(path)
  img = image_preprocessing(path)
  embed = face_embedding(facenet_model, img)
  pred = get_prediction(expand_dims(embed, axis = 0))
  result.append(pred[0])
result

test/23.jpeg
test/19.jpeg
test/16.jpeg
test/25.jpeg
test/6.jpeg
test/7.jpeg
test/17.jpeg
test/22.jpeg
test/14.jpeg
test/3.jpeg
test/15.jpeg
test/1.jpg
test/11.jpg
test/21.jpeg
test/4.jpeg
test/24.jpeg
test/18.jpeg
test/2.jpg
test/12.jpeg
test/9.jpeg
test/20.jpeg
test/8.jpeg
test/13.jpeg
test/5.jpeg
test/10.jpeg


[4, 2, 2, 4, 0, 0, 2, 4, 1, 3, 1, 3, 4, 4, 3, 4, 2, 3, 1, 0, 4, 0, 1, 3, 1]

In [ ]:
dir = 'nonsense'
result = []
for file in listdir(dir):
  path = dir + '/' + file
  print(path)
  img = image_preprocessing(path)
  embed = face_embedding(facenet_model, img)
  pred = get_prediction(expand_dims(embed, axis = 0))
  result.append(pred[0])
result

nonsense/3.jpg
nonsense/4.jpg
nonsense/1.jpg
nonsense/2.jpg


[5, 3, 5, 5]

In [ ]:
face = [5,5,5,5]
acc = accuracy_score(result, face)
print("Accuracy score {0} %.".format(acc*100))

Accuracy score 75.0 %.


In [ ]:
# preprocess = Pipeline(steps = [('face_transform',face_arr_transform(facenet_model))])
#                                #('normalize', Normalizer(norm = 'l2'))])
# X, y = preprocess.fit_transform(X, y)
# print(X.shape, len(y))
# print(X[1])
# print(y[:5])
# X, y = oversample.fit_resample(X, y)
# print(X.shape, y.shape)
# print(X[1])
# print(y[:5])

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# RF_classifier = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=3, max_features=1)
# RF_classifier.fit(train_face, train_name)
# yhat_train = my_model.predict(train_face)
# yhat_test = my_model.predict(test_face)
# train_accuracy = accuracy_score(train_name, yhat_train)
# test_accuracy = accuracy_score(test_name, yhat_test)
# print("Training accuracy of the model is {0} %.".format(train_accuracy*100))
# print("Testing accuracy of the model is {0} %.".format(test_accuracy*100))

In [ ]:
# class face_arr_transform(BaseEstimator, TransformerMixin):
#   def __init__(self, model):
#     self.model = model
  
#   def fit(self, X, y = None):
#     return self
  
#   def face_embedding(self, face):
#     face = face.astype('float32')
#     mean, std = face.mean(), face.std()
#     face = (face-mean)/std
#     per = expand_dims(face, axis=0)
#     yhat = self.model.predict(per)
#     return yhat[0]

#   def transform(self, X, y = None):
#     new_X = list()
#     for face in X:
#       embed = self.face_embedding(face)
#       new_X.append(embed)
#     return asarray(new_X), y
